In [8]:
# import zipfile

# # Ruta del archivo ZIP
# zip_file_path = '/content/tesis-main.zip'

# # Ruta donde quieres extraer los archivos
# extract_to_path = '/content/'

# # Descomprimir el archivo
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_to_path)

# print("Archivos extraídos con éxito.")

Archivos extraídos con éxito.


* Carga el video utilizando cv2.VideoCapture.
* Extrae los frames del video uno por uno.
* Preprocesa cada frame para que sean compatibles con el modelo de PyTorch (tamaño, normalización, etc.).
* Usa un modelo preentrenado de PyTorch (como ResNet) para extraer características de cada frame.
* Aplica un algoritmo de clustering (como K-Means) a las características extraídas.
* Asigna cada frame al cluster correspondiente.
* Visualiza los resultados mostrando los frames y su cluster correspondiente.

In [36]:
import numpy as np

class SimilarityClustering:
    def __init__(self, threshold):
        self.threshold = threshold
        self.labels_ = None

    def fit(self, features):
        cosine_similarities = np.array([cosine_similarity(features[i], features[i + 1])
                                        for i in range(len(features) - 1)])

        cosine_similarities = np.insert(cosine_similarities, 0, cosine_similarity(features[0], features[0]))

        self.labels_ = label_clusters(cosine_similarities, self.threshold)

cosine_similarity = lambda vec1, vec2: np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def label_clusters(cosine_similarities, threshold):
    clusters = np.zeros(len(cosine_similarities), dtype=int)
    current_cluster = 0
    for i in range(1, len(cosine_similarities)):
        print(i,cosine_similarities[i])

        if cosine_similarities[i] < threshold:
            current_cluster += 1
        clusters[i] = current_cluster
    print(clusters)
    return clusters

def clustering_function(model):
    def get_labels(features):
        model.fit(features)
        labels = model.labels_
        return labels
    return get_labels


In [37]:
import torch
from sklearn.cluster import AgglomerativeClustering
import warnings
from models.resnet18 import get_model as get_resnet18
from preprocessing.transforms import BASELINE
from clustering.metrics import eval_massive_cluster
from clustering.model import clustering_function
from experiment_framework import experiment

warnings.filterwarnings('ignore')


if torch.cuda.is_available():
 dev = "cuda:0"
else:
 dev = "cpu"


device = torch.device(dev)
epochs = 10
steps=1
clustering_eval_function = lambda cluster_labels, tags: eval_massive_cluster(cluster_labels, tags, steps, epochs)


EXPERIMENT_DEFINITIONS = [
   {
      "name": "ResnetAglomerative video.mp4",
      "model": get_resnet18(device),
      "preprocessing": BASELINE,
      "path": "video.mp4",
      "grouper_function": clustering_function(AgglomerativeClustering(None, distance_threshold=50)),
      "evaluation_function": clustering_eval_function,
      "show": True,
      "device": device
   },
   {
      "name": "ResnetAglomerative video2.mp4",
      "model": get_resnet18(device),
      "preprocessing": BASELINE,
      "path": "video2.mp4",
      "grouper_function": clustering_function(AgglomerativeClustering(None, distance_threshold=50)),
      "evaluation_function": clustering_eval_function,
      "show": True,
      "device": device
   },
      {
      "name": "SimilarityClustering video.mp4",
      "model": get_resnet18(device),
      "preprocessing": BASELINE,
      "path": "video2.mp4",
      "grouper_function": clustering_function(SimilarityClustering(threshold=0.999)),
      "evaluation_function": clustering_eval_function,
      "show": True,
      "device": device
   },
      {
      "name": "SimilarityClustering video2.mp4",
      "model": get_resnet18(device),
      "preprocessing": BASELINE,
      "path": "video2.mp4",
      "grouper_function": clustering_function(SimilarityClustering(threshold=0.999)),
      "evaluation_function": clustering_eval_function,
      "show": True,
      "device": device
   },
]

for exp_definition in EXPERIMENT_DEFINITIONS:
   experiment(**exp_definition)


Experiment name: ResnetAglomerative video.mp4
Precision - Mean: 0.69, std: 0.0281367746
Recall - Mean: 0.63, std: 0.0339345169
F1-Score - Mean: 0.61, std: 0.0304841227
Accuracy - Mean: 0.63, std: 0.0339345169

Experiment name: ResnetAglomerative video2.mp4
Precision - Mean: 0.71, std: 0.0453087181
Recall - Mean: 0.73, std: 0.0476690618
F1-Score - Mean: 0.70, std: 0.0478856333
Accuracy - Mean: 0.73, std: 0.0476690618
1 0.9901678
2 0.9844674
3 0.99233615
4 0.9969075
5 0.99971074
6 0.99991447
7 0.9999472
8 0.9998191
9 0.9998092
10 0.99995524
11 0.99689907
12 0.9994184
13 0.99967176
14 0.9995555
15 0.9996139
16 0.99883544
17 0.9998647
18 0.99994165
19 0.99940276
20 0.9999812
21 0.99997705
22 0.99995434
23 0.99998564
24 0.9999851
25 0.9999894
26 0.999993
27 0.99999946
28 0.9999992
29 0.9999998
30 0.9999999
31 0.99999994
32 0.99999994
33 0.9999993
34 0.99999976
35 0.99999976
36 0.9999998
37 0.9999998
38 0.99999994
39 1.0000001
40 1.0000001
41 1.0000001
42 1.0000001
43 1.0000001
44 1.0000001